In [3]:
!pip install pandas pyarrow scikit-learn -q


[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [8]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet

--2023-06-12 15:30:13--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:2550:6600:b:20a5:b140:21, 2600:9000:2550:fc00:b:20a5:b140:21, 2600:9000:2550:8800:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:2550:6600:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1730999 (1.7M) [binary/octet-stream]
Saving to: ‘green_tripdata_2023-03.parquet’

green_tripdata_2023 100%[===================>]   1.65M  --.-KB/s    in 0.1s    

2023-06-12 15:30:13 (13.3 MB/s) - ‘green_tripdata_2023-03.parquet’ saved [1730999/1730999]



In [5]:
import pandas as pd


df = pd.read_parquet('yellow_tripdata_2022-01.parquet')

Q1) Total number of columns  
A: 19

In [6]:
len(df.columns)

19

Q2) Standard deviation of trip duration, in minutes  
A: 46.45 min

In [7]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
df.duration.std()

46.44530513776802

Q3) Outliers, fraction of records left after dropping
A: 98%

In [9]:
df_clean = df[(df.duration >= 1) & (df.duration <= 60)]
df_clean.shape[0] / df.shape[0]

0.9827547930522406

Q4) One-hot encoding, dimensionality of matrix  
A: 515 columns

In [10]:
categorical = ['PULocationID', 'DOLocationID']
df_clean[categorical] = df_clean[categorical].astype(str)

/var/folders/w5/103dbchj5sb_529bq4km5drc0000gp/T/ipykernel_77471/1058706765.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[categorical] = df_clean[categorical].astype(str)


In [11]:
df_train = df_clean[categorical]
train_dicts = df_train.to_dict(orient='records')
type(train_dicts)

list

In [12]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [13]:
cols = X_train.shape[1]
cols

515

Q5) Training model, RMSE_train  
A: 6.99

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

target = 'duration'
y_train = df_clean[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

rmse = mean_squared_error(y_train, y_pred, squared=False)
rmse


6.9861908315536585

Q6) Evaluating the model, RMSE_validation  
A: 7.79

In [15]:
from typing import Tuple, Optional, Any
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

# Hard time finding the correct types being returned


def process_data(parquet_filename: str, vectorizer=None) -> Tuple[Any, Any, Optional[DictVectorizer]]:
    df = pd.read_parquet(parquet_filename)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df_clean = df[(df.duration >= 1) & (df.duration <= 60)]
    categorical = ['PULocationID', 'DOLocationID']
    df_clean[categorical] = df_clean[categorical].astype(str)
    df_train = df_clean[categorical]
    train_dicts = df_train.to_dict(orient='records')
    
    if not vectorizer:
    # dictionary vectorizer not provided, treat as training job
        vectorizer = DictVectorizer()
        X_train = vectorizer.fit_transform(train_dicts)
    else:
    # dictionary vectorizer provided, treat as validation job
        X_train = vectorizer.transform(train_dicts)
        

    target = 'duration'
    y_train = df_clean[target].values
    
    return X_train, y_train, vectorizer
    

In [16]:
X_train, y_train, dv_train = process_data(parquet_filename='yellow_tripdata_2022-01.parquet')
X_test, y_test, _ = process_data(parquet_filename='yellow_tripdata_2022-02.parquet', vectorizer=dv_train)

/var/folders/w5/103dbchj5sb_529bq4km5drc0000gp/T/ipykernel_77471/2423312881.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[categorical] = df_clean[categorical].astype(str)
/var/folders/w5/103dbchj5sb_529bq4km5drc0000gp/T/ipykernel_77471/2423312881.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[categorical] = df_clean[categorical].astype(str)


In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

7.786409148550689